# Lesson 1 - Building a QA Agent with Google Gemini

```mermaid
graph LR
    %% User / Client Layer
    User([User / A2A Client])
    
    %% Main Orchestrator Layer (Lesson 8)
    subgraph OrchestratorLayer [Router/Requirement Agent]
        Concierge["<b>Healthcare Concierge Agent</b><br/>(BeeAI Framework)<br/><code>Port: 9996</code>"]
    end

    subgraph SubAgents [A2A Agent Servers]
        direction TB

        PolicyAgent["<b>Policy Agent</b><br/>(Gemini with A2A SDK)<br/><code>Port: 9999</code>"]
        ResearchAgent["<b>Research Agent</b><br/>(Google ADK)<br/><code>Port: 9998</code>"]

        ProviderAgent["<b>Provider Agent</b><br/>(LangGraph + LangChain)<br/><code>Port: 9997</code>"]
    end

    %% Data & Tools Layer
    subgraph DataLayer [Data Sources & Tools]
        PDF["Policy PDF"]
        Google[Google Search Tool]
        MCPServer["FastMCP Server<br/>(<code>doctors.json</code>)"]
    end
    
    Label_UA["Sends Query - A2A"]
    Label_CP["A2A"]
    Label_CR["A2A"]
    Label_CProv["A2A"]
    Label_MCP["MCP (stdio)"]

    %% -- CONNECTIONS --
    
    User --- Label_UA --> Concierge

    Concierge --- Label_CP --> PolicyAgent
    Concierge --- Label_CR --> ResearchAgent
    Concierge --- Label_CProv --> ProviderAgent
    
    PolicyAgent -- "Reads" --> PDF
    ResearchAgent -- "Calls" --> Google
    
    ProviderAgent --- Label_MCP --> MCPServer

    classDef orchestrator fill:#f9f,stroke:#333,stroke-width:2px;
    classDef agent fill:#e1f5fe,stroke:#0277bd,stroke-width:2px;
    classDef tool fill:#fff3e0,stroke:#ef6c00,stroke-width:1px,stroke-dasharray: 5 5;
    
    classDef protocolLabel fill:#ffffff,stroke:none,color:#000;
    
    class Concierge orchestrator;
    class PolicyAgent,ResearchAgent,ProviderAgent agent;
    class PDF,Google,MCPServer tool;
    
    class Label_UA,Label_CP,Label_CR,Label_CProv,Label_MCP protocolLabel;
```

In this lesson, you will build a basic Question Answering (QA) agent using [Google Gemini](https://deepmind.google/models/gemini/) via the [Google Gen AI SDK](https://github.com/googleapis/python-genai). You will use **[LiteLLM](https://www.litellm.ai/)** to interact with the model to analyze a PDF document containing health insurance policy details. Then, you will refactor this logic into a reusable Python class, laying the groundwork for the next lesson where you will wrap this agent in an [Agent2Agent (A2A)](https://a2a-protocol.org/) server.

## 1.1. Import Libraries and Setup

First, import the necessary libraries. You will use `litellm` to interact with the LLM and standard libraries to handle file encoding.

In [8]:
import base64
from pathlib import Path

import litellm
from IPython.display import Markdown, display

from helpers import setup_env

ModuleNotFoundError: No module named 'a2a.types'

In [ ]:
setup_env()

In [7]:
pip install a2a


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/46.7 kB ? eta -:--:--
     ---------------------------------------- 46.7/46.7 kB 1.2 MB/s eta 0:00:00
  Using cached cffi-2.0.0-cp311-cp311-win_amd64.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/331.7 kB ? eta -:--:--
   ---------------------------------- ----- 286.7/331.7 kB 8.9 MB/s eta 0:00:01
   ---------------------------------------  327.7/331.7 kB 6.9 MB/s eta 0:00:01
   ---------------------------------------  327.7/331.7 kB 6.9 MB/s eta 0:00:01
   ---------------------------------------- 331.7/331.7 kB 2.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.0 MB ? 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sop-deutils 1.0.2.1 requires cryptography==42.0.7, but you have cryptography 46.0.5 which is incompatible.
sop-deutils 1.0.2.1 re

## 1.2. Load and Encode Data

You read the insurance policy PDF (`2026AnthemgHIPSBC.pdf`) and encode it in base64 so it can be passed to the model as context.


In [ ]:
with Path("data/2026AnthemgHIPSBC.pdf").open("rb") as file:
    pdf_data = base64.standard_b64encode(file.read()).decode("utf-8")

## 1.3. Query the Model

Now you will send a specific query to the model: "How much would I pay for mental health therapy?". You provide the model with a system instruction to act as an expert insurance agent and pass the PDF document alongside the user's text prompt.


In [ ]:
prompt = "How much would I pay for mental health therapy?"

In [ ]:
response = litellm.completion(
    model="gemini/gemini-3-flash-preview",
    # For Vertex AI:
    # model="vertex_ai/gemini-3-flash-preview",
    reasoning_effort="minimal",
    max_tokens=1000,
    messages=[
        {
            "role": "system",
            "content": "You are an expert insurance agent designed to assist with coverage queries. Use the provided documents to answer questions about insurance policies. If the information is not available in the documents, respond with 'I don't know'",
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:application/pdf;base64,{pdf_data}"},
                },
            ],
        },
    ],
)

In [ ]:
response_text = response.choices[0].message.content.replace("$", r"\\$")
display(Markdown(response_text))

## 1.4. Refactor into an Agent Class

To make this code reusable and easier to integrate into an A2A server later, we have wrapped the logic into a `PolicyAgent` class in a file named `policy_agent.py`. This class initializes the data in the `__init__` method and exposes an `answer_query` method.

In [ ]:
from IPython.display import Code, display

display(Code("policy_agent.py"))

## 1.5. Test the Agent Class

Finally, import the `PolicyAgent` class you just created and test it with the same query to ensure it works as expected.

In [ ]:
from policy_agent import PolicyAgent

print("Running Health Insurance Policy Agent")
agent = PolicyAgent()
prompt = "How much would I pay for mental health therapy?"

response = agent.answer_query(prompt)
display(Markdown(response))

## 1.6. Resources

- [Google Gemini API Documentation](https://ai.google.dev/docs)
- [LiteLLM Documentation](https://docs.litellm.ai/docs/)